In [1]:
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.auth import exceptions
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
import pandas as pd

# Ruta al archivo JSON con las credenciales de la cuenta de servicio
KEY_FILE_LOCATION = 'C:\\Users\\ehmet\\OneDrive\\Escritorio\\Proyectos de data\\data_analytics\\data-analytics-project-409620-26c1f804bc93.json'  # Actualiza la ruta

# ID de la propiedad de Google Analytics 4
PROPERTY_ID = '421423594'  # Reemplaza con tu identificador de propiedad

# Configurar las credenciales
try:
    credentials = service_account.Credentials.from_service_account_file(
        KEY_FILE_LOCATION,
        scopes=['https://www.googleapis.com/auth/analytics.readonly']
    )
except exceptions.GoogleAuthError as auth_error:
    print(f"Error de autenticación: {auth_error}")
    raise

# Crear un cliente de Google Analytics Data API
client = BetaAnalyticsDataClient(credentials=credentials)

'''
Consulta de ejemplo que funciona
try:
    # Realizar una consulta de ejemplo
    response = client.run_report(
        request=RunReportRequest(
            property=f'properties/{PROPERTY_ID}',
            date_ranges=[DateRange(start_date='2023-01-01', end_date='2023-01-07')],
            dimensions=[Dimension(name='date')],
            metrics=[Metric(name='engagedSessions')],
        )
    )

    # Imprimir la respuesta
    print(response)

except Exception as e:
    # Imprimir información de error
    print(f"Error: {e}")
'''

# Definir las nuevas fechas de inicio y fin
start_date = '2024-01-01'
end_date = '2024-01-08'

# Configurar la solicitud del informe con las nuevas fechas
response = client.run_report(
    request=RunReportRequest(
        property=f'properties/{PROPERTY_ID}',
        date_ranges=[DateRange(start_date=start_date, end_date=end_date)],
        dimensions=[Dimension(name='date')],
        metrics=[Metric(name='engagedSessions')],
    )
)


# Bloque de codigo obtiene un dataframe con filas que representan dias, y 3 columnas: Fecha, visitas y usuarios
# Procesar la respuesta y organizar en un DataFrame
data = []
for row in response.rows:
    dimensions = [dimension.value for dimension in row.dimension_values]
    metrics = [metric.value for metric in row.metric_values]
    data.append(dimensions + metrics)

# Crear DataFrame
columns = ['Fecha', 'Visitas', 'Usuarios']
df = pd.DataFrame(data, columns=columns)

# Imprimir el DataFrame
df.head()

,Fecha,Visitas,Usuarios
